In [171]:
# Import
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import warnings
import copy

In [172]:
fn_en = '/mnt/permanent/home/eszti/dipterv/panlex/data/smith/smith_all_data/en_orig.txt'
fn_it = '/mnt/permanent/home/eszti/dipterv/panlex/data/smith/smith_all_data/it.txt'
limit = None
encoding = 'latin-1'

In [173]:
class EmbeddingModel():
    def __init__(self):
        self.syn0 = None
        self.index2word = None

    def normalize(self):
        self.syn0 /= np.sqrt((self.syn0 ** 2).sum(1))[:, None]
        print('normalized, # > 0.001: {}'.format(
            len(np.where(abs(np.linalg.norm(self.syn0, axis=1) - 1) > 0.0001)[0]) ))

    def _read(self, fn, limit=None, lexicon=None, encoding='utf-8'):
        raise NotImplementedError

    def read(self, fn, limit=None, lexicon=None, encoding='utf-8'):
        print('Reading embedding from {}'.format(fn))
        self._read(fn, limit, lexicon, encoding)
        self.normalize()
        print('Syn0 size: {0}'.format(self.syn0.shape))

    def get(self, word):
        if word not in self.index2word:
            raise ValueError('Out of dictionary word: {}'.format(word))
        else:
            idx = self.index2word.index(word)
            return self.syn0[idx]

class TextEmbedding(EmbeddingModel):
    def __init__(self):
        EmbeddingModel.__init__(self)

    def _read(self, fn, limit=None, lexicon=None, encoding='utf-8'):
        id2row = []
        def filter_lines(f):
            for i,line in enumerate(f):
                if limit is not None and i > limit:
                    break
                word = line.split()[0]
                if i != 0 and (lexicon is None or word in lexicon):
                    id2row.append(word)
                    yield line

        #get the number of columns
        with open(fn, encoding=encoding) as f:
            f.readline()
            ncols = len(f.readline().split())

        with open(fn, encoding=encoding) as f:
            m = np.matrix(np.loadtxt(filter_lines(f),
                          comments=None, usecols=range(1,ncols)))
        self.syn0 = np.asarray(m)
        self.index2word = id2row

In [174]:
emb_mod_it = TextEmbedding()
emb_mod_it.read(fn=fn_it, limit=limit, encoding=encoding)

Reading embedding from /mnt/permanent/home/eszti/dipterv/panlex/data/smith/smith_all_data/it.txt
normalized, # > 0.001: 0
Syn0 size: (200000, 300)


In [175]:
emb_mod_en = TextEmbedding()
emb_mod_en.read(fn=fn_en, limit=limit, encoding=encoding)

Reading embedding from /mnt/permanent/home/eszti/dipterv/panlex/data/smith/smith_all_data/en_orig.txt


/home/eszti/.virtualenvs/dipterv3/lib/python3.5/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


normalized, # > 0.001: 0
Syn0 size: (200000, 300)


/home/eszti/.virtualenvs/dipterv3/lib/python3.5/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


In [176]:
def print_num(num):
    print(emb_mod_en.index2word[num])
    print(emb_mod_en.syn0[num])
    
def check_nans(emb):
    nans = np.where(np.isnan(emb.syn0).any(axis=1))[0]
    print(len(nans))
    print(nans)
    for nan in nans:
        print_num(nan)
    print('new embedding: ')
    ok_emb = copy.deepcopy(emb)
    ok_emb.syn0 = np.delete(ok_emb.syn0, nans, axis=0)
    ok_emb.index2word = [w for i, w in enumerate(emb.index2word) if i not in nans]
    print(len(ok_emb.syn0))
    print(len(ok_emb.index2word))
    return ok_emb

In [177]:
ok_en = check_nans(emb_mod_en)

5
[ 32150 122170 140745 141044 154961]
----------------------------------------------------------------------------------------------------
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  na

In [178]:
ok_it = check_nans(emb_mod_it)

0
[]
new embedding: 
200000
200000


In [179]:
emb_mod_en.get('dog')

array([  1.08404936e-01,   3.49647595e-02,  -3.17763692e-02,
         7.17696988e-02,  -1.59557295e-02,   3.49183854e-03,
         6.48673445e-02,   5.09712318e-03,   3.12064377e-02,
         2.43393681e-02,  -1.18621025e-01,   2.40745650e-02,
        -2.25438822e-02,  -6.82019142e-02,   8.55176159e-02,
        -5.20579997e-02,   1.19639576e-01,  -1.35551956e-01,
         3.72209624e-02,  -4.51522850e-02,  -5.43713302e-02,
         5.12414116e-02,   9.99608090e-02,  -4.38548171e-02,
         5.32613751e-02,   1.34717557e-01,  -3.10001063e-03,
        -1.07683112e-01,   3.46992843e-02,   1.55135622e-01,
        -7.16346089e-02,  -1.17208966e-02,   5.10579313e-02,
         3.42153803e-02,   1.18997731e-01,   6.08327941e-02,
         6.24495042e-02,   7.21453966e-02,  -7.15418606e-02,
        -9.52494650e-02,   6.26386973e-03,   4.86273213e-02,
         4.91532310e-03,   8.08264345e-02,   1.19021926e-01,
        -3.20794813e-02,  -7.62182561e-03,  -2.82563029e-02,
         6.91260366e-02,

In [180]:
ok_en.get('dog')

array([  1.08404936e-01,   3.49647595e-02,  -3.17763692e-02,
         7.17696988e-02,  -1.59557295e-02,   3.49183854e-03,
         6.48673445e-02,   5.09712318e-03,   3.12064377e-02,
         2.43393681e-02,  -1.18621025e-01,   2.40745650e-02,
        -2.25438822e-02,  -6.82019142e-02,   8.55176159e-02,
        -5.20579997e-02,   1.19639576e-01,  -1.35551956e-01,
         3.72209624e-02,  -4.51522850e-02,  -5.43713302e-02,
         5.12414116e-02,   9.99608090e-02,  -4.38548171e-02,
         5.32613751e-02,   1.34717557e-01,  -3.10001063e-03,
        -1.07683112e-01,   3.46992843e-02,   1.55135622e-01,
        -7.16346089e-02,  -1.17208966e-02,   5.10579313e-02,
         3.42153803e-02,   1.18997731e-01,   6.08327941e-02,
         6.24495042e-02,   7.21453966e-02,  -7.15418606e-02,
        -9.52494650e-02,   6.26386973e-03,   4.86273213e-02,
         4.91532310e-03,   8.08264345e-02,   1.19021926e-01,
        -3.20794813e-02,  -7.62182561e-03,  -2.82563029e-02,
         6.91260366e-02,